# Topic Modeling of Product Reviews on Amazon Scraped using Selenium in Python
> A tutorial demonstrating scraping product reviews from Amazon and extracting and analysing the topics from the text data.

- toc: true
- comments: true
- categories: [python, text analytics, web scraping, topic modeling, selenium, gensim, nlp]

## Introduction

The blog covers the step-by-step process to scrap product reviews from Amazon webpage and analysing main topics from the extracted data. We will scrap 1000 reviews from the Amazon for Apple iPhone 11 64GB. With this data, we will convert each review doc into bag-of words for applying the topic modeling algorithm. We will be using Latent Dirichlet Allocation (LDA) algorithm in this tutorial. The main python libraries used are:

- selenium: Selenium is a portable framework for testing web applications. We will be using this to interact with the browser and open URLs.
- gensim: Gensim is an open-source library for unsupervised topic modeling and natural language processing, using modern statistical machine learning.

## Web Scraping

Web scraping is a technique for extracting information from the internet automatically using a software that simulates human web surfing.Web scraping helps us extract large volumes of data about customers, products, people, stock markets, etc. It is usually difficult to get this kind of information on a large scale using traditional data collection methods. We can utilize the data collected from a website such as e-commerce portal, social media channels to understand customer behaviors and sentiments, buying patterns, and brand attribute associations which are critical insights for any business.

The first and foremost thing while scraping a website is to understand the structure of the website. We will be scraping the reviews for Apple iPhone 11 64GB on Amazon.in website. We will scrape 1000 reviews from different users across multiple pages. We will scrape user name, date of review and review and export it into a csv file for any further analysis.

### Import Packages

- Install selenium package (if not already worked with before) using command '!pip install selenium'
- Import webdriver from selenium in the notebook which we use to open an instance of Chrome browser.
- The executable file for launching Chrome 'chromedriver.exe' should be in the same folder as the notebook.

In [4]:
#Importing packages
from selenium import webdriver
import pandas as pd

### Script for Scraping

The below code opens the new chrome browser window and open our website with the url link provided. By the way, chrome knows that you are accessing it through an automated software!
![](img/chrome.png)

In [114]:
driver = webdriver.Chrome('chromedriver.exe')
url = 'https://www.amazon.in/Apple-iPhone-11-64GB-White/product-reviews/B07XVMCLP7/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber1'
driver.get(url)

We will inspect 3 items (user id, date and comment) on our web page and understand how we can extract them.
- Xpath for User id: Inspecting the userid, we can see the highlighted text represents the XML code for user id.The XML path (XPath)for the userid is shown below:
    
    //*[@id="customer_review-RBOIMRTKIYBBR"]/div[1]/a/div[2]/span
    
![](img/chrome-2.png)
    
There is an interesting thing to note here that the XML path contains a review id, which uniquely denotes each review on the website. This will be very helpful as we try to recursively scrape multiple comments.

- Xpath for Date & review: Similarily, we will find the XPaths for date and review. 
- Selenium has a function called “find_elements_by_xpath”. We will pass our XPath into this function and get a selenium element. Once we have the element, we can extract the text inside our XPath using the ‘text’ function.
- We will recursively run the code for different review id and extract user id, date and review for each review id. Also, we will recursively go to next pages by simply changing the page numbers in the url to extract more comments until we get the desired number of comments.

In [14]:
driver = webdriver.Chrome('chromedriver.exe')

#Creating empty data frame to store user_id, dates and comments from ~5K users.
data = pd.DataFrame(columns = ['date','username','review'])

j = 1
while (j<=130):
    # Running while loop only till we get 1K reviews
    if (len(data)<1000):
        url = 'https://www.amazon.in/Apple-iPhone-11-64GB-White/product-reviews/B07XVMCLP7/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=' + str(j)
        driver.get(url)
        ids = driver.find_elements_by_xpath("//*[contains(@id,'customer_review-')]")
        review_ids = []
        for i in ids:
            review_ids.append(i.get_attribute('id'))

        for x in review_ids:
            #Extract dates from for each user on a page
            date_element = driver.find_elements_by_xpath('//*[@id="' + x +'"]/span')[0]
            date = date_element.text

            #Extract user ids from each user on a page
            username_element = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div[1]/a/div[2]/span')[0]
            username = username_element.text

            #Extract Message for each user on a page
            review_element = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div[4]')[0]
            review = review_element.text
            
           #Adding date, userid and comment for each user in a dataframe    
            data.loc[len(data)] = [date,username,review]
        j=j+1
    else:
        break

### Data Cleaning

- We perform few data cleaning operations such as replacing line breaks with a space and copy the data into .csv file which can be used for further analysis.

In [15]:
import copy
data = copy.deepcopy(data)

def remove_space(s):
    return s.replace("\n"," ")

data['review'] = data['review'].apply(remove_space)
data.to_csv('amazon_reviews.csv', header=True, sep=',')

In [16]:
data = pd.read_csv('amazon_reviews.csv',index_col=[0])
data

,date,username,review
0,Reviewed in India on 20 October 2019,Suman Biswas,May be my first negative review about the prod...
1,Reviewed in India on 17 September 2019,Kaushik Bajaj,It's very expensive but the quality you get is...
2,Reviewed in India on 29 September 2019,Sunny Kumar,The iPhone design is good and the camera quali...
3,Reviewed in India on 30 September 2019,shanu Kumar,Awesome Phone. Nice upgrade from iPhone 6s to ...
4,Reviewed in India on 14 October 2019,Amazon Customer,My Phone is Producing Too Much Heat Even Didn’...
...,...,...,...
995,Reviewed in India on 1 March 2020,Amazon Customer,❤️
996,Reviewed in India on 9 March 2020,Chirag Patel,Ok
997,Reviewed in India on 11 March 2020,chintu,Excellent
998,Reviewed in India on 8 March 2020,Amazon Customer,Excellent


- Since the goal of further analysis is to perform topic modeling, we will solely focus on the review text, and drop other metadata columns i.e. date and user name.

In [17]:
# Remove the columns
data = data.drop(columns=['date', 'username'], axis=1)
# Print out the data
data

,review
0,May be my first negative review about the prod...
1,It's very expensive but the quality you get is...
2,The iPhone design is good and the camera quali...
3,Awesome Phone. Nice upgrade from iPhone 6s to ...
4,My Phone is Producing Too Much Heat Even Didn’...
...,...
995,❤️
996,Ok
997,Excellent
998,Excellent


## Topic Modeling using LDA

Topic modeling is a type of statistical modeling for discovering the abstract “topics” that occur in a collection of documents. Latent Dirichlet Allocation (LDA) is an example of topic model and is used to classify text in a document to a particular topic. LDA is a generative probabilistic model that assumes each topic is a mixture over an underlying set of words, and each document is a mixture of over a set of topic probabilities.

Illustration of LDA input/output workflow (Credit: http://chdoig.github.io/pytexas2015-topic-modeling/#/3/4)
![](img/lda-1.png)

### Data Pre-processing

We will preprocess the review data using gensim library. Few of the actions performed by preprocess_string as follows:
- Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
- All stopwords are removed.
- Words are lemmatized: words in third person are changed to first person and verbs in past and future tenses are changed into present. 
- Words are stemmed: words are reduced to their root form.

Please see below the output after pre-processing for one of the reviews.

In [19]:
import gensim
from gensim.parsing.preprocessing import preprocess_string

# print unprocessed text
print(data.review[0])

# print processed text
print(preprocess_string(data.review[0]))

May be my first negative review about the product & Amazon both. I was much elated to receive the iPhone 11 so fast, next day of dispatch i.e. 28/09/19, but the thing I got started heating up every now and then. Contacted Applecare, just to be consoled that it's quite normal. As it continued, tried to return the product by speaking to Amazon customer support but in vain. Some body called me back to convey that only Apple will decide which one to take back. Why is then Amazon took up the sacred duty of selling such an item which they can't exchange/ have no control ? The product developed new issues like proximity sensor malfunction and last but most importantly loosing mobile network every other minute(even had two software updates). It was handed over to the Apple ASP as the return window closed on 10/10/19 (what use it was for??) and diagnosed as having issues and has further been sent to Apple repair facility at Bengaluru. So I'm here w/out my first iPhone after using it(suffering f

In [20]:
processed_data = data['review'].map(preprocess_string)
processed_data

0      [neg, review, product, amazon, elat, receiv, i...
1                                [expens, qualiti, osum]
2      [iphon, design, good, camera, qualiti, awesom,...
3      [awesom, phone, nice, upgrad, iphon, iphon, lo...
4      [phone, produc, heat, didn’t, sim, half, hour,...
                             ...                        
995                                                   []
996                                                   []
997                                              [excel]
998                                              [excel]
999                                           [flawless]
Name: review, Length: 1000, dtype: object

### Preparign Document-Term-Matrix

Gensim requires that tokens be converted to a dictionary. In this instance a dictionary is a mapping between words and their integer IDs. We then create a  Document-Term-Matrix where we use Bag-of-Words approach returning the vector of word and its frequency (number of occurences in the document) for each document.

In [44]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our list of documents (corpus), where every unique term is assigned an index. 
dictionary = corpora.Dictionary(processed_data)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in processed_data]

### Running LDA Model

We will now run the LDA Model. The number of topics you give is largely a guess/arbitrary. The model assumes the document contains that many topics. However, finding the number of topics explaining the data is a optimisation problem and can be found by 'Coherence Model'.

Here, we have used number of topics = 3

In [71]:
#RUN THE MODEL
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
TOPIC_CNT= 3
ldamodel = Lda(doc_term_matrix, num_topics=TOPIC_CNT, id2word = dictionary, passes=50)

We can then see the weights of top 20 words in each topic, which can help us to explain the topic.

In [72]:
#Results
topics= ldamodel.print_topics(num_topics=TOPIC_CNT, num_words=20)
topics

[(0,
  '0.066*"phone" + 0.047*"good" + 0.017*"appl" + 0.015*"best" + 0.014*"charger" + 0.010*"time" + 0.010*"work" + 0.010*"issu" + 0.009*"charg" + 0.008*"iphon" + 0.008*"camera" + 0.007*"screen" + 0.007*"get" + 0.007*"purchas" + 0.006*"mobil" + 0.006*"android" + 0.006*"batteri" + 0.006*"heat" + 0.006*"us" + 0.006*"like"'),
 (1,
  '0.043*"iphon" + 0.038*"batteri" + 0.037*"camera" + 0.036*"phone" + 0.026*"awesom" + 0.026*"good" + 0.024*"best" + 0.024*"qualiti" + 0.024*"life" + 0.023*"great" + 0.011*"displai" + 0.010*"amaz" + 0.010*"upgrad" + 0.009*"perform" + 0.009*"love" + 0.008*"superb" + 0.008*"bui" + 0.008*"dai" + 0.007*"face" + 0.006*"better"'),
 (2,
  '0.064*"product" + 0.030*"amazon" + 0.028*"appl" + 0.023*"nice" + 0.015*"phone" + 0.015*"bui" + 0.015*"monei" + 0.013*"excel" + 0.011*"deliveri" + 0.010*"got" + 0.010*"time" + 0.010*"iphon" + 0.009*"devic" + 0.009*"perfect" + 0.009*"worth" + 0.009*"valu" + 0.009*"thank" + 0.009*"happi" + 0.008*"receiv" + 0.007*"good"')]

### Extracting Topics

We can identify the follow topics emerging out of reviews of Amazon iPhone 11 64GB on Amazon:

- Topic #1: There seems to discussion of heat/ charging issue with the product.
- Topic #2: The discussion on iPhone's features such as camera, display, battery.
- Topic #3: iPhone being value for money and discussuin on Amazon delivery.

In [111]:
word_dict = {};
for i in range(TOPIC_CNT):
    words = ldamodel.show_topic(i, topn = 20)
    word_dict['Topic #' + '{:2d}'.format(i+1)] = [i[0] for i in words]
pd.DataFrame(word_dict)

,Topic # 1,Topic # 2,Topic # 3
0,phone,iphon,product
1,good,batteri,amazon
2,appl,camera,appl
3,best,phone,nice
4,charger,awesom,phone
5,time,good,bui
6,work,best,monei
7,issu,qualiti,excel
8,charg,life,deliveri
9,iphon,great,got


The below code provide the % of topics a document is about. This helps to find the dominant topic in each review. 

In [74]:
doc_to_topic = []
for i in range(len(doc_term_matrix)):
    top_topics = ldamodel.get_document_topics(doc_term_matrix[i], minimum_probability=0.0)
    topic_vec = [top_topics[j][1] for j in range(TOPIC_CNT)]
    doc_to_topic.append(topic_vec)

In [106]:
#Dataframe of topic
document_topics = pd.DataFrame(doc_to_topic)
document_topics = document_topics.rename(columns=lambda x: x + 1)
document_topics.columns = document_topics.columns.astype(str)
document_topics = document_topics.rename(columns=lambda x: 'Topic #' + x)

In [112]:
#Dataframe of review and topics
data_new = pd.concat([data,document_topics],axis=1,join='inner')
data_new.head()

,review,Topic #1,Topic #2,Topic #3
0,May be my first negative review about the prod...,0.004879,0.004542,0.990579
1,It's very expensive but the quality you get is...,0.084246,0.831019,0.084735
2,The iPhone design is good and the camera quali...,0.696215,0.133451,0.170334
3,Awesome Phone. Nice upgrade from iPhone 6s to ...,0.036537,0.802536,0.160927
4,My Phone is Producing Too Much Heat Even Didn’...,0.567121,0.010595,0.422285


### Visualization of LDA Model

There is a nice way to visualize the LDA mode using the package pyLDAvis. This visualization allows us to compare topics on two reduced dimensions and observe the distribution of words in topics. The size of the bubble measures the importance of the topics, relative to the data.

In [77]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\61920959\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


## Endnotes

I hope this blog helps in understanding how powerful Topic Modeling is in understanding unstructured textual data.  Feel free to play around with the code by opening in Colab or cloning the repo in github.

If you have any comments or suggestions please comment below or reach out to me at - [Twitter](https://twitter.com/rahulsingla0959) or [LinkedIn](https://www.linkedin.com/in/rahul-singla1/)